In [1]:
#@title Copyright 2022 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_asset_from_cloud_geotiff.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_asset_from_cloud_geotiff.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Cloud GeoTiff Backed Earth Engine Assets

***Note:*** *The REST API contains new and advanced features that may not be suitable for all users.  If you are new to Earth Engine, please get started with the [JavaScript guide](https://developers.google.com/earth-engine/guides/getstarted).*

Earth Engine can load images from Cloud Optimized GeoTiffs (COGs) in Google Cloud Storage ([learn more](https://developers.google.com/earth-engine/guides/image_overview#images-from-cloud-geotiffs)).  This notebook demonstrates how to create Earth Engine assets backed by COGs.  An advantage of COG-backed assets is that the spatial and metadata fields of the image will be indexed at asset creation time, making the image more performant in collections.  (In contrast, an image created through `ee.Image.loadGeoTIFF` and put into a collection will require a read of the GeoTiff for filtering operations on the collection.)  A disadvantage of COG-backed assets is that they may be several times slower than standard assets when used in computations.

To create a COG-backed asset, make a `POST` request to the Earth Engine [`CreateAsset` endpoint](https://developers.google.com/earth-engine/reference/rest/v1alpha/projects.assets/create).  As shown in the following, this request must be authorized to create an asset in your user folder.

In [69]:
import pandas as pd
from google.cloud import storage
import os
import json
import subprocess
import re

## Start an authorized session

To be able to make an Earth Engine asset in your user folder, you need to be able to authenticate as yourself when you make the request.  You can use credentials from the Earth Engine authenticator to start an [`AuthorizedSession`](https://google-auth.readthedocs.io/en/master/reference/google.auth.transport.requests.html#google.auth.transport.requests.AuthorizedSession).  You can then use the `AuthorizedSession` to send requests to Earth Engine.

In [3]:
import ee
from google.auth.transport.requests import AuthorizedSession
!{'earthengine authenticate'}
#ee.Authenticate()  #  or !earthengine authenticate --auth_mode=gcloud
session = AuthorizedSession(ee.data.get_persistent_credentials())

Fetching credentials using gcloud
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=6LiDnPlsfd9MicgBb9rieV6s0WbOMu&access_type=offline&code_challenge=zOdfq1s4BYoUFUSImfQ9hzUEOehmbUHYDZb1ntt4KyY&code_challenge_method=S256


Credentials saved to file: [/Users/christiannilsen/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update


Successfully saved authorization token.


## Get image information from gcp
Get list of images and their respective uris from the gcp data bucket


In [4]:
def list_blobs_with_prefix(bucket_name, prefix, delimiter=None):
    

    storage_client = storage.Client(project='swhm-prod')

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    # Note: The call returns a response only when the iterator is consumed.
    blob_list = []
    for blob in blobs:
        blob_list.append(blob.name)

    if delimiter:
        print("Prefixes:")
        for prefix in blobs.prefixes:
            blob_list.append([prefix])
    
    return blob_list


In [5]:
BUCKET_NAME = 'live_data_layers'
blobsout = list_blobs_with_prefix(BUCKET_NAME,'raster')

In [6]:
df = pd.DataFrame(blobsout, columns=['blob_name'])
df['uri'] = 'gs://'+BUCKET_NAME+'/'+df['blob_name']
(df)

,blob_name,uri
0,rasters/Age_of_Imperviousness.tif,gs://live_data_layers/rasters/Age_of_Imperviou...
1,rasters/Flow_Duration_Index.tif,gs://live_data_layers/rasters/Flow_Duration_In...
2,rasters/HSPF_Land_Cover_Type.tif,gs://live_data_layers/rasters/HSPF_Land_Cover_...
3,rasters/Hydrologic_Response_Units.tif,gs://live_data_layers/rasters/Hydrologic_Respo...
4,rasters/Imperviousness.tif,gs://live_data_layers/rasters/Imperviousness.tif
5,rasters/Land_Cover.tif,gs://live_data_layers/rasters/Land_Cover.tif
6,rasters/Land_Cover.tif.tif,gs://live_data_layers/rasters/Land_Cover.tif.tif
7,rasters/Land_Cover_rio.tif,gs://live_data_layers/rasters/Land_Cover_rio.tif
8,rasters/Land_Use.tif,gs://live_data_layers/rasters/Land_Use.tif
9,rasters/Land_Use_rio.tif,gs://live_data_layers/rasters/Land_Use_rio.tif


In [7]:

file_names = [os.path.splitext(os.path.basename(file_path))[0] for file_path in df['blob_name']]
df["asset_name"] = file_names

pretty_names  = [os.path.splitext(os.path.basename(file_path))[0].replace('_', ' ') for file_path in df['blob_name']]

df["pretty_name"] = pretty_names
df

,blob_name,uri,asset_name,pretty_name
0,rasters/Age_of_Imperviousness.tif,gs://live_data_layers/rasters/Age_of_Imperviou...,Age_of_Imperviousness,Age of Imperviousness
1,rasters/Flow_Duration_Index.tif,gs://live_data_layers/rasters/Flow_Duration_In...,Flow_Duration_Index,Flow Duration Index
2,rasters/HSPF_Land_Cover_Type.tif,gs://live_data_layers/rasters/HSPF_Land_Cover_...,HSPF_Land_Cover_Type,HSPF Land Cover Type
3,rasters/Hydrologic_Response_Units.tif,gs://live_data_layers/rasters/Hydrologic_Respo...,Hydrologic_Response_Units,Hydrologic Response Units
4,rasters/Imperviousness.tif,gs://live_data_layers/rasters/Imperviousness.tif,Imperviousness,Imperviousness
5,rasters/Land_Cover.tif,gs://live_data_layers/rasters/Land_Cover.tif,Land_Cover,Land Cover
6,rasters/Land_Cover.tif.tif,gs://live_data_layers/rasters/Land_Cover.tif.tif,Land_Cover.tif,Land Cover.tif
7,rasters/Land_Cover_rio.tif,gs://live_data_layers/rasters/Land_Cover_rio.tif,Land_Cover_rio,Land Cover rio
8,rasters/Land_Use.tif,gs://live_data_layers/rasters/Land_Use.tif,Land_Use,Land Use
9,rasters/Land_Use_rio.tif,gs://live_data_layers/rasters/Land_Use_rio.tif,Land_Use_rio,Land Use rio


## Build the manifest

In [8]:
def get_layer_dict(asset): 
    data = json.load(open("data/rasters.json"))
    layer_dict = data[asset]
    print(asset)
    #delete dictionaries 
    try: 
        del layer_dict["layer"]
        del layer_dict["values"]
        del layer_dict["labels"]

    except KeyError:
        pass
    

    layer_dict['pretty_name'] = asset
    return layer_dict

In [9]:
get_layer_dict('Precipitation mm')

Precipitation mm


{'discrete': 'FALSE',
 'sourceName': 'Salathé et al 2019',
 'sourceUrl': 'https://cig.uw.edu/our-work/applied-research/heavy-precip-and-stormwater/',
 'units': 'mm/year',
 'scale': 2500,
 'default_reduction': 'mean',
 'description': 'Mean annual precipitation (1970-1999)',
 'safe_name': 'precipitation_mm',
 'pretty_name': 'Precipitation mm'}

Create json_data

In [10]:
df['json_data'] = 'na'
for index, row in df.iterrows():
    key_value = row['pretty_name']
    try: 
        json_value = json.dumps(get_layer_dict(key_value))
    except KeyError:
        json_value = "NA"
    df.at[index, 'json_data'] = json_value


Age of Imperviousness
Flow Duration Index
HSPF Land Cover Type
Hydrologic Response Units
Imperviousness
Land Cover
Land Use
Population Density
Precipitation mm
Runoff mm
Slope
Slope Categories
Soils
Total Copper Concentration
Total Kjeldahl Nitrogen Concentration
Total Phosphorus Concentration
Total Suspended Solids Concentration
Total Zinc Concentration
Traffic


In [11]:
df

,blob_name,uri,asset_name,pretty_name,json_data
0,rasters/Age_of_Imperviousness.tif,gs://live_data_layers/rasters/Age_of_Imperviou...,Age_of_Imperviousness,Age of Imperviousness,"{""discrete"": ""TRUE"", ""sourceName"": ""The Nature..."
1,rasters/Flow_Duration_Index.tif,gs://live_data_layers/rasters/Flow_Duration_In...,Flow_Duration_Index,Flow Duration Index,"{""discrete"": ""FALSE"", ""sourceName"": ""The Natur..."
2,rasters/HSPF_Land_Cover_Type.tif,gs://live_data_layers/rasters/HSPF_Land_Cover_...,HSPF_Land_Cover_Type,HSPF Land Cover Type,"{""discrete"": ""TRUE"", ""sourceName"": ""The Nature..."
3,rasters/Hydrologic_Response_Units.tif,gs://live_data_layers/rasters/Hydrologic_Respo...,Hydrologic_Response_Units,Hydrologic Response Units,"{""discrete"": ""TRUE"", ""sourceName"": ""The Nature..."
4,rasters/Imperviousness.tif,gs://live_data_layers/rasters/Imperviousness.tif,Imperviousness,Imperviousness,"{""discrete"": ""FALSE"", ""sourceName"": ""The Natur..."
5,rasters/Land_Cover.tif,gs://live_data_layers/rasters/Land_Cover.tif,Land_Cover,Land Cover,"{""discrete"": ""TRUE"", ""sourceName"": ""The Nature..."
6,rasters/Land_Cover.tif.tif,gs://live_data_layers/rasters/Land_Cover.tif.tif,Land_Cover.tif,Land Cover.tif,NA
7,rasters/Land_Cover_rio.tif,gs://live_data_layers/rasters/Land_Cover_rio.tif,Land_Cover_rio,Land Cover rio,NA
8,rasters/Land_Use.tif,gs://live_data_layers/rasters/Land_Use.tif,Land_Use,Land Use,"{""discrete"": ""TRUE"", ""sourceName"": ""Puget Soun..."
9,rasters/Land_Use_rio.tif,gs://live_data_layers/rasters/Land_Use_rio.tif,Land_Use_rio,Land Use rio,NA


In [12]:
import requests

def send_request(session, asset_name,props):
    # Earth Engine enabled Cloud Project.
    project_folder = 'ee-swhm'
    # A folder (or ImageCollection) name and the new asset name.
    asset_id = asset_name

    url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId=production_layers/{}'
    #url = 'https://earthengine.googleapis.com/v1beta/projects/{}/assets?assetId=production_layers/{}'
    params = {'overwrite': True}
    response = session.post(
      url = url.format(project_folder, asset_id),
      data = props, 
        params=params
    )
    
    print(json.loads(response.content))
    
    


In [16]:
import json
import pprint
def process_df(df): 
    for index, row in df.iterrows():
        json_string = row['json_data']
        print(row['pretty_name'])
        print('...')
        if json_string is not None and json_string != 'NA':
            layer = df['pretty_name'][index]
            asset_name = df['asset_name'][index]
            uri = df['uri'][index]

            # Request body as a dictionary.
            request = {
                'type': 'IMAGE',
                'gcs_location': {'uris': uri},
                'properties':json.loads(df['json_data'][index])
            }

            props = json.dumps(request)
            send_request(session, asset_name,props)


## Send the request

Make the POST request to the Earth Engine [`projects.assets.create`](https://developers.google.com/earth-engine/reference/rest/v1beta/projects.assets/create) endpoint.

In [23]:
#process_df(df)

index = 0
layer = df['pretty_name'][index]
asset_name = df['asset_name'][index]
uri = df['uri'][index]
ee_cmd = f'earthengine upload image \
--asset_id=projects/ee-swhm/assets/production_layers/{asset_name} \
{uri}'

!{ee_cmd}

Started upload task with ID: O2ILZHBJT3QTTRWGDFAZCTTH


In [48]:
uri

'gs://live_data_layers/rasters/Age_of_Imperviousness.tif'

In [71]:
def get_type(file_path): 
    command = ["rio", "cogeo", "info",file_path]
    result = subprocess.run(command, capture_output=True,text=True)
    pattern = r"Dtype:\s+(\w+)"
    # find the Dtype value
    match = re.search(pattern, s)
    if match:
        dtype = match.group(1)
        return(dtype)
    else:
        return("Dtype not found.")


In [90]:
index = 0
layer = df['pretty_name'][index]
asset_name = df['asset_name'][index]
uri = df['uri'][index]

if (get_type(uri) == 'uint8'): 
    pyramiding_policy = "sample" 
else: 
    pyramiding_policy = "mean"
    
    
    
ee_cmd = f'earthengine upload image \
--asset_id=projects/ee-swhm/assets/production_layers/{asset_name} \
--pyramiding_policy={pyramiding_policy} \
{uri}'
ee_cmd    

'earthengine upload image --asset_id=projects/ee-swhm/assets/production_layers/Age_of_Imperviousness --pyramiding_policy=sample gs://live_data_layers/rasters/Age_of_Imperviousness.tif'

In [88]:
#if need to remove it first 
#ee_cmd = f'earthengine rm projects/ee-swhm/assets/production_layers/Age_of_Imperviousness'

In [91]:
!{ee_cmd}






Started upload task with ID: MWPX57I7LUT5YSCWVHJOPZEG


True

In [56]:
#print(result.stdout)
output_dict = ast.literal_eval(result.stdout)

print(output_dict)


ValueError: malformed node or string: b"Driver: GTiff\nFile: gs://live_data_layers/rasters/Age_of_Imperviousness.tif\nCOG: True\nCompression: LZW\nColorSpace: None\n\nProfile\n    Width:            31523\n    Height:           30154\n    Bands:            1\n    Tiled:            True\n    Dtype:            uint8\n    NoData:           None\n    Alpha Band:       False\n    Internal Mask:    False\n    Interleave:       BAND\n    ColorMap:         False\n    ColorInterp:      ('gray',)\n    Scales:           (1.0,)\n    Offsets:          (0.0,)\n\nGeo\n    Crs:              EPSG:3857\n    Origin:           (-13904050.342789244, 6328206.664228426)\n    Resolution:       (15.0, -15.0)\n    BoundingBox:      (-13904050.342789244, 5875896.664228426, -13431205.342789244, 6328206.664228426)\n    MinZoom:          6\n    MaxZoom:          13\n\nImage Metadata\n    AREA_OR_POINT: Area\n\nImage Structure\n    COMPRESSION: LZW\n    INTERLEAVE: BAND\n    LAYOUT: COG\n\nBand 1\n    ColorInterp: gr

In [25]:
cmd = f'rio cogeo info {uri}'
!{cmd}

Driver: GTiff
File: gs://live_data_layers/rasters/Age_of_Imperviousness.tif
COG: True
Compression: LZW
ColorSpace: None

Profile
    Width:            31523
    Height:           30154
    Bands:            1
    Tiled:            True
    Dtype:            uint8
    NoData:           None
    Alpha Band:       False
    Internal Mask:    False
    Interleave:       BAND
    ColorMap:         False
    ColorInterp:      ('gray',)
    Scales:           (1.0,)
    Offsets:          (0.0,)

Geo
    Crs:              EPSG:3857
    Origin:           (-13904050.342789244, 6328206.664228426)
    Resolution:       (15.0, -15.0)
    BoundingBox:      (-13904050.342789244, 5875896.664228426, -13431205.342789244, 6328206.664228426)
    MinZoom:          6
    MaxZoom:          13

Image Metadata
    AREA_OR_POINT: Area

Image Structure
    COMPRESSION: LZW
    INTERLEAVE: BAND
    LAYOUT: COG

Band 1
    ColorInterp: gray

IFD
    Id      Size           BlockSize     Decimation           
    0  

In [13]:
#direct upload from cmd line 
for index, row in df.iterrows():
    json_string = row['json_data']
    print(row['pretty_name'])
    print('...')
    if json_string is not None and json_string != 'NA':
        layer = df['pretty_name'][index]
        asset_name = df['asset_name'][index]
        uri = df['uri'][index]
    
        

,blob_name,uri,asset_name,pretty_name,json_data
0,rasters/Age_of_Imperviousness.tif,gs://live_data_layers/rasters/Age_of_Imperviou...,Age_of_Imperviousness,Age of Imperviousness,"{""discrete"": ""TRUE"", ""sourceName"": ""The Nature..."
1,rasters/Flow_Duration_Index.tif,gs://live_data_layers/rasters/Flow_Duration_In...,Flow_Duration_Index,Flow Duration Index,"{""discrete"": ""FALSE"", ""sourceName"": ""The Natur..."
2,rasters/HSPF_Land_Cover_Type.tif,gs://live_data_layers/rasters/HSPF_Land_Cover_...,HSPF_Land_Cover_Type,HSPF Land Cover Type,"{""discrete"": ""TRUE"", ""sourceName"": ""The Nature..."
3,rasters/Hydrologic_Response_Units.tif,gs://live_data_layers/rasters/Hydrologic_Respo...,Hydrologic_Response_Units,Hydrologic Response Units,"{""discrete"": ""TRUE"", ""sourceName"": ""The Nature..."
4,rasters/Imperviousness.tif,gs://live_data_layers/rasters/Imperviousness.tif,Imperviousness,Imperviousness,"{""discrete"": ""FALSE"", ""sourceName"": ""The Natur..."
5,rasters/Land_Cover.tif,gs://live_data_layers/rasters/Land_Cover.tif,Land_Cover,Land Cover,"{""discrete"": ""TRUE"", ""sourceName"": ""The Nature..."
6,rasters/Land_Cover.tif.tif,gs://live_data_layers/rasters/Land_Cover.tif.tif,Land_Cover.tif,Land Cover.tif,NA
7,rasters/Land_Cover_rio.tif,gs://live_data_layers/rasters/Land_Cover_rio.tif,Land_Cover_rio,Land Cover rio,NA
8,rasters/Land_Use.tif,gs://live_data_layers/rasters/Land_Use.tif,Land_Use,Land Use,"{""discrete"": ""TRUE"", ""sourceName"": ""Puget Soun..."
9,rasters/Land_Use_rio.tif,gs://live_data_layers/rasters/Land_Use_rio.tif,Land_Use_rio,Land Use rio,NA


PATCH https://earthengine.googleapis.com/v1/{asset.name=projects/*/assets/**}